In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from utils.model import *
import Levenshtein
from torch.nn import functional as F
from timeit import default_timer as timer
from utils import util
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler
from utils.nn.lookahead import Lookahead
from utils.nn.radam import RAdam

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

###################################################################################################
import torch.cuda.amp as amp
class AmpNet(Net):
    @torch.cuda.amp.autocast()
    def forward(self, *args):
        return super(AmpNet, self).forward(*args)

is_mixed_precision = True #False  #



###################################################################################################

STOI = {
    '<sos>': 190,
    '<eos>': 191,
    '<pad>': 192,
}

image_size = 224
vocab_size = 193
max_length = 300



###################################################################################################
def do_valid(net, tokenizer, valid_loader):

    valid_probability = []
    valid_truth = []
    valid_length = []
    valid_num = 0

    net.eval()
    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        batch_size = len(batch['index'])
        image  = batch['image' ].cuda()
        token  = batch['token' ].cuda()
        length = batch['length']

        with torch.no_grad():
            logit = net(image, token, length)
            probability = F.softmax(logit,-1)

        valid_num += batch_size
        valid_probability.append(probability.data.cpu().numpy())
        valid_truth.append(token.data.cpu().numpy())
        valid_length.extend(length)
        print('\r %8d / %d  %s'%(valid_num, len(valid_loader.sampler),time_to_str(timer() - start_timer,'sec')),end='',flush=True)

    assert(valid_num == len(valid_loader.sampler)) #len(valid_loader.dataset))

    #----------------------
    probability = np.concatenate(valid_probability)
    predict = probability.argmax(-1)
    truth   = np.concatenate(valid_truth)
    length  = valid_length


    #----
    p = probability[:,:-1].reshape(-1,vocab_size)
    t = truth[:,1:].reshape(-1)

    non_pad = np.where(t!=STOI['<pad>'])[0] #& (t!=STOI['<sos>'])
    p = p[non_pad]
    t = t[non_pad]
    loss = util.np_loss_cross_entropy(p, t)

    #----
    lb_score = 0
    if 1:
        score = []
        for p, t in zip(predict, truth):
            t = truth[1][1:length[1]-1]
            p = predict[1][1:length[1]-1]
            t = tokenizer.one_sequence_to_text(t)
            p = tokenizer.one_sequence_to_text(p)
            s = Levenshtein.distance(p, t)
            score.append(s)
        lb_score = np.mean(score)

    return [loss, lb_score]

def message(mode='print'):
    if mode == ('print'):
        asterisk = ' '
        loss = batch_loss
    if mode == ('log'):
        asterisk = '*' if iteration % iter_save == 0 else ' '
        loss = train_loss

    text = \
        '%0.5f  %5.4f%s %4.2f  | ' % (rate, iteration / 10000, asterisk, epoch,) + \
        '%4.3f  %5.2f  | ' % (*valid_loss,) + \
        '%4.3f  %4.3f  | ' % (*loss,) + \
        '%s' % (time_to_str(timer() - start_timer, 'min'))

    return text

def time_to_str(t, mode='min'):
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)

    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)

    else:
        raise NotImplementedError




fold = 0
out_dir = '../model/res_lstm_att/fold{}'.format(fold)
initial_checkpoint = None

start_lr = 0.001
batch_size = 64


## setup  ----------------------------------------
for f in ['checkpoint']: 
    os.makedirs(out_dir + '/' + f, exist_ok=True)

## dataset ------------------------------------

df_train = pd.read_pickle('../data/df_train0.pkl')
df_valid = pd.read_pickle('../data/df_valid0.pkl')

# 加载数据
tokenizer = util.load_tokenizer()
train_dataset = util.BmsDataset(df_train,tokenizer)
valid_dataset = util.BmsDataset(df_valid,tokenizer)

train_loader = DataLoader(
    train_dataset,
    sampler = RandomSampler(train_dataset),
    batch_size=batch_size,
    drop_last=True,
    num_workers=8,
    pin_memory=True,
    collate_fn=util.collate_fn,
)
valid_loader = DataLoader(
    valid_dataset,
    # sampler=SequentialSampler(valid_dataset),
    sampler=util.FixNumSampler(valid_dataset, 5000),
    batch_size=32,
    drop_last=False,
    num_workers=4,
    pin_memory=True,
    collate_fn=util.collate_fn,
)

print('train_dataset : \n%s\n' % (train_dataset))
print('valid_dataset : \n%s\n' % (valid_dataset))

## net ----------------------------------------
print('** net setting **\n')
if is_mixed_precision:
    scaler = amp.GradScaler()
    net = AmpNet().cuda()
else:
    net = Net().cuda()

if initial_checkpoint is not None:
    f = torch.load(initial_checkpoint, map_location=lambda storage, loc: storage)
    start_iteration = f['iteration']
    start_epoch     = f['epoch']

    state_dict = f['state_dict']
    #decoder_state_dict = { k.replace('image_pos','image_pos1'):v for k,v, in decoder_state_dict.items()}
    net.load_state_dict(state_dict, strict=False)  # True
    if 0:
        net.encoder.load_state_dict(f['encoder_state_dict'], strict=True)  # True

        decoder_state_dict = f['decoder_state_dict']
        decoder_state_dict = {k.replace('gate.', 'attention.gate.'): v for k, v, in decoder_state_dict.items()}
        net.load_state_dict(decoder_state_dict, strict=False)  # True

else:
    start_iteration = 0
    start_epoch = 0

print('\tinitial_checkpoint = %s\n' % initial_checkpoint)
print('\n')

# -----------------------------------------------
if 0:  ##freeze
    for p in encoder.parameters(): p.requires_grad = False

# 定义优化器
optimizer = Lookahead(RAdam(filter(lambda p: p.requires_grad, net.parameters()), lr=start_lr), alpha=0.5, k=5)
# optimizer = RAdam(filter(lambda p: p.requires_grad, net.parameters()),lr=start_lr)

# 最大迭代 batchsize 次数
num_iteration = 80000 * 1000
# 每 iter_log 打印
iter_log = 1000
# 每 iter_valid 次验证
iter_valid = 1000
# 每 iter_save 次保存
iter_save = 1000 # 1*1000

print('optimizer\n  %s\n' % (optimizer))
print('\n')

## start training here! ##############################################
print('** start training here! **\n')
print('   is_mixed_precision = %s \n' % str(is_mixed_precision))
print('   batch_size = %d\n' % (batch_size))
print('                      |----- VALID ---|---- TRAIN/BATCH --------------\n')
print('rate     iter   epoch | loss  lb(lev) | loss0  loss1  | time          \n')
print('----------------------------------------------------------------------\n')
            # 0.00000   0.00* 0.00  | 0.000  0.000  | 0.000  0.000  |  0 hr 00 min

# ----
valid_loss = np.zeros(2, np.float32)
train_loss = np.zeros(2, np.float32)
batch_loss = np.zeros_like(train_loss)
sum_train_loss = np.zeros_like(train_loss)
sum_train = 0
loss0 = torch.FloatTensor([0]).cuda().sum()
loss1 = torch.FloatTensor([0]).cuda().sum()
#loss2 = torch.FloatTensor([0]).cuda().sum()

start_timer = timer()
iteration = start_iteration
epoch = start_epoch
rate = 0
while iteration < num_iteration:

    for t, batch in enumerate(train_loader):

        if (iteration % iter_save == 0):
            if iteration != start_iteration:
                torch.save({
                    'state_dict': net.state_dict(),
                    'iteration': iteration,
                    'epoch': epoch,
                }, out_dir + '/checkpoint/{}_model.pth'.format(iteration))

        if (iteration % iter_valid == 0):
            if iteration != start_iteration:
                valid_loss = do_valid(net, tokenizer, valid_loader)

        if (iteration % iter_log == 0):
            if iteration != start_iteration:
                print('\r', end='', flush=True)
                print(message(mode='log') + '\n')

        # learning rate schduler ------------
        rate = util.get_learning_rate(optimizer)

        # one iteration update  -------------
        batch_size = len(batch['index'])
        image  = batch['image'].cuda()
        token  = batch['token'].cuda()
        length = batch['length']

        # ----
        net.train()
        optimizer.zero_grad()

        if is_mixed_precision:
            # https://pytorch.org/docs/master/amp.html
            # https://pytorch.org/docs/master/notes/amp_examples.html#amp-examples
            with amp.autocast():
                #assert(False)
                logit = net(image, token, length)
                loss0 = seq_cross_entropy_loss(logit, token, length)

            scaler.scale(loss0).backward()
            # scaler.scale(loss1).backward()
            # scaler.scale(loss0 + loss1 + loss2).backward()
            # scaler.unscale_(optimizer)
            # torch.nn.utils.clip_grad_norm_(net.parameters(), 2)
            scaler.step(optimizer)
            scaler.update()


        else:
            logit = net(image, token, length)
            loss0 = seq_cross_entropy_loss(logit, token, length)
            (loss0).backward()
            optimizer.step()

        # print statistics  --------
        epoch += 1 / len(train_loader)
        iteration += 1

        # batch_loss = np.array([loss0.item(), loss1.item(), loss2.item()])
        batch_loss = np.array([loss0.item(), loss1.item()])
        sum_train_loss += batch_loss
        sum_train += 1
        # 求最近 100 个 batch 的平均 loss
        if iteration % 100 == 0:
            train_loss = sum_train_loss / (sum_train + 1e-12)
            sum_train_loss[...] = 0
            sum_train = 0

        print('\r', end='', flush=True)
        print(message(mode='print'), end='', flush=True)

print('\n')

# main #################################################################
if __name__ == '__main__':
    pass


len(tokenizer) : vocab_size 193
train_dataset : 
	len = 2020155
	df  = (2020155, 6)


valid_dataset : 
	len = 404031
	df  = (404031, 6)


** net setting **

<All keys matched successfully>
	initial_checkpoint = None



optimizer
  Lookahead (
Parameter Group 0
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    step_counter: 0
    weight_decay: 0
)



** start training here! **

   is_mixed_precision = True 

   batch_size = 64

                      |----- VALID ---|---- TRAIN/BATCH --------------

rate     iter   epoch | loss  lb(lev) | loss0  loss1  | time          

----------------------------------------------------------------------

0.00100  0.0863  0.03  | 0.000   0.00  | 1.634  0.000  |  0 hr 03 min